# Homework 4 Step 2: Levenshtein Distance with Dynamic Programming

For the next part of the assignment, we will use dynamic programming to compute the similarity of protein sequences from the NCBI database.

A protein sequence, at its core, is a chain of amino acids. We will represent these protein sequences in the homework as lists of characters corresponding to amino acids (for example, Methionine is represented by M and Lysine by K). For instance, this is the sequence for the **50S ribosomal protein L36** found in Bacillus Anthracis (commonly known as anthrax):

`[‘M’, ‘K’, ‘V’, ‘R’, ‘P’, ‘S’, ‘V’, ‘K’, ‘P’, ‘I’, ‘C’, ‘E’, ‘K’, ‘C’, ‘K’, ‘V’, ‘I’, ‘R’, ‘R’, ‘R’, ‘G’, ‘K’, ‘M’, ‘V’, ‘I’, ‘C’, ‘E’, ‘N’, ‘P’, ‘K’, ‘H’, ‘K’, ‘Q’, ‘K’, ‘Q’, ‘G’]`

Amino acids with similar biochemical properties can be grouped into classes. Below is a table showing all the amino acids, their one letter code, and the class they belong to.

| **Class** | **Name of amino acids** |  
| :--- | :--- |
|Aliphatic |Glycine(G), Alanine(A), Valine(V), Leucine(L), Isoleucine(I)|  
|Hydroxyl or Sulfur/Selenium-containing|Serine(S), Cysteine(C), Threonine(T), Methionine(M)|  
|Cyclic|Proline(P)|  
|Aromatic|Phenylalanine(F), Tyrosine(Y), Tryptophan(W)|  
|Basic|Histidine(H), Lysine(K), Arginine(R)|  
|Acidic and their Amide|Aspartic Acid(D), Glutamic Acid(E), Asparagine(N), Glutamine(Q)|  

## Step 2.1 Substitution Cost

In this section, we’ll do an introduction to dynamic programming using a genetics application. A common question in genetics is to find _homologs_, where a gene is inherited by two species from a common ancestor.  During this process, the two species’ genes may slightly mutate, so they may not exactly match, but they are likely to be very similar -- thus bioinformaticists use approximate string matching, either between the genes or between the proteins they produce, to look for candidate homologs.

You will implement a modified version of the classic Levenshtein distance algorithm to measure the edit distance between two strings. We will be comparing protein sequences from the NCBI database using edit distance as a naïve proxy for homology. Recall from class that the Levenshtein distance is given by:

$$\text{lev}_{a,b}(i,j) = \begin{cases} \max(i,j) &, \text{if}\min(i,j) = 0\\ \min \begin{cases} \text{lev}_{a,b}(i-1,j) + 1\\ \text{lev}_{a,b}(i,j-1) + 1\\ \text{lev}_{a,b}(i-1,j-1) + 1_{(a_i \neq b_j)}\end{cases} &, \text{otherwise.}\end{cases}$$

Insertions and deletions have a cost of 1. We’ll add a slight modification to the Levenshtein distance where **substitutions** -- here, replacements of an amino acid in a protein with another amino acid -- now fall into two categories: _conservative_ and _nonconservative_. Replacing an amino acid with another one that belongs in the same class is a conservative substitution and has a cost of 1. A nonconservative substitution has a cost of 2. In addition, ‘X’ is a wildcard character and replacing it also has a cost of 1.

Write a function `modifiedSubstitutionCost(a,b)` that takes two proteins (as single character strings) and returns the cost of substituting one for the other. Hint: you may want to create a dictionary with protiens as keys and integer classes as values.

In [1]:
# Your modifiedSubstitutionCost(a,b) and any supporting data structures go here.
try:
    protein_dict = {'G':1,'A':1,'V':1,'L':1,'I':1,
                    'S':2,'C':2,'T':2,'M':2,
                    'P':3,
                    'F':4,'Y':4,'W':4,
                    'H':5,'K':5,'R':5,
                    'D':6,'E':6,'N':6,'Q':6
                   }
    def modifiedSubstitutionCost(a, b):
        if a == 'X' or b == 'X':
            if a != b:
                cost = 1
            else:
                cost = 0
  
        else:
            if a != b:
                if protein_dict[a] != protein_dict[b]:
                    cost = 2
                if protein_dict[a] == protein_dict[b]:
                    cost = 1
            else:
                cost = 0


        return cost
# YOUR CODE HERE
except:
    raise NotImplementedError()

In [2]:
assert(modifiedSubstitutionCost('G','G') == 0)


In [3]:
assert(modifiedSubstitutionCost('X','X') == 0)


In [4]:
assert(modifiedSubstitutionCost('F','Y') == 1)


## Step 2.2 Implementing Levenshtein

The dataset we will be using is contained in the `proteins.fasta` file. Please note that the format is a standard in bioinformatics but may be a bit different than what you’re used to seeing. 
The first line is the description of the protein and begins with a `>` character. The subsequent line(s) contain(s) the actual sequence data. For example:

`> NCBIAccessionNumber ProteinName`  
$\hspace{3ex}$`MNIKGSPWKGSLLLLLVSNLLLCQ...`

For this data set, we’ll be using the Prolactin precursor (Homo Sapiens) protein as the reference sequence:

In [5]:
referenceSequence = ['M','N','I','K','G','S','P','W','K','G','S','L','L','L','L','L','V','S','N','L','L','L','C','Q','S','V','A','P','L','P','I','C','P','G','G','A','A','R','C','Q','V','T','L','R','D','L','F','D','R','A','V','V','L','S','H','Y','I','H','N','L','S','S','E','M','F','S','E','F','D','K','R','Y','T','H','G','R','G','F','I','T','K','A','I','N','S','C','H','T','S','S','L','A','T','P','E','D','K','E','Q','A','Q','Q','M','N','Q','K','D','F','L','S','L','I','V','S','I','L','R','S','W','N','E','P','L','Y','H','L','V','T','E','V','R','G','M','Q','E','A','P','E','A','I','L','S','K','A','V','E','I','E','E','Q','T','K','R','L','L','E','G','M','E','L','I','V','S','Q','V','H','P','E','T','K','E','N','E','I','Y','P','V','W','S','G','L','P','S','L','Q','M','A','D','E','E','S','R','L','S','A','Y','Y','N','L','L','H','C','L','R','R','D','S','H','K','I','D','N','Y','L','K','L','L','K','C','R','I','I','H','N','N','N','C',]

Implement a function `levenshtein(s1, s2)` that returns the modified Levenshtein distance (a number) between two lists of characters using dynamic programming.

In [6]:
# Your levenshtein(s1, s2) function goes here.
import numpy as np
try:
    def levenshtein(s1,s2):
        if len(s1) < len(s2):
            return levenshtein(s2,s1)
        if len(s2) == 0:
            return len(s1)

        memo = np.zeros((len(s1) +1, len(s2) +1))
        memo[0] = range(len(s2) + 1 )
        for i, c1 in enumerate(s1):
            memo[i + 1,0] = i + 1
            for j, c2 in enumerate(s2):
                insertions = memo[i,j + 1] + 1
                deletions = memo[i + 1,j] + 1
                sub_cost = modifiedSubstitutionCost(c1, c2)
                substitutions = memo[i,j] + sub_cost
                memo[i + 1, j + 1] = min(insertions, deletions, substitutions)
        return memo[-1,-1]
# YOUR CODE HERE
except:
    raise NotImplementedError()

In [7]:
import time
str2 = ['X','I','D','N','Y','L','K','L','L','K','C','R','I','I','H','N','N','N','C']

start_time = time.time()
print(levenshtein(referenceSequence,str2));
print("--- %s seconds ---" % (time.time() - start_time))


209.0
--- 0.01015925407409668 seconds ---


In [8]:
# Don't delete this cell


In [9]:
# Don't delete this cell


## Step 2.3 Find the top k-homologs for Prolactin precursor

Write a function `output_k_homologs(fastaFilename, reference_sequence, k)` that returns a DateFrame containing the top `k` homologs in `fastaFilename` for the given `reference_sequence`. You should include the columns `proteinName` and `levenshteinDistance`.

In [38]:
import pandas as pd
# Your output_k_homologs(fastaFilename, reference_sequence, k) function goes here

try:
    def output_k_homologs(fastaFilename, reference_sequence, k):
        file = open(fastaFilename, 'r').read()
        #concatenate all items into one single string
        string_file = ''.join(file)
        fix_file = string_file.split('>')
        fix_file = fix_file[1:len(fix_file)]

        #creating list of protein name
        f1 = []
        for i in fix_file:
            x = i.rstrip()
            f1.append(x)

        f2 = []
        name_lst = []
        for i in f1:
            name,protein = i.split('\n',1)
            name_lst.append(name)
            f2.append(protein)

        protein_sequence = []
        for i in f2:
            protein = i.replace('\n','')
            protein_sequence.append(protein)
            
        lst1 = []
        for i in name_lst:
            _,name = i.split(' ',1)
            lst1.append(name)
            
        # list of protein names
        protein_names = []
        for i in lst1:
            name,_ = i.split(' [',2)
            protein_names.append(name)
    
        # create a dataframe
        df = pd.DataFrame({'proteinName':protein_names,'proteinSequence':protein_sequence})

        #create a function to apply to the dataframe for levenshtein 
        def transform_func(protein_name):
            char_lst = list(protein_name)
            score = levenshtein(char_lst,reference_sequence)
            return score

        df['levenhsteinDistance'] = df['proteinSequence'].apply(transform_func)
        df = df.sort_values(by = ['levenhsteinDistance'], ascending = False).reset_index()
        return df.head(k)


# YOUR CODE HERE
except:
    raise NotImplementedError()

In [39]:
homologs = output_k_homologs("proteins.fasta", referenceSequence, 10)
display(homologs)


index          proteinName  \
0    399  glycoside hydrolase   
1    411  glycoside hydrolase   
2    406  glycoside hydrolase   
3    409  glycoside hydrolase   
4    410  glycoside hydrolase   
5    400  glycoside hydrolase   
6    415  glycoside hydrolase   
7    405  glycoside hydrolase   
8    404  glycoside hydrolase   
9    550  glycoside hydrolase   

                                     proteinSequence  levenhsteinDistance  
0  MLKRSITVVLCVLLLIAPNAAFAYKNPASLPDEWGQYGLGDPYVLK...               1219.0  
1  MLKRMIMAILCALLLIAPQAAFAYKNPKSLPDEWGQYGLGDPYVMK...               1217.0  
2  MKKILSLVIVISLVLTVPLSALAYSNPFEFPDSWKWDTGVFYGEGD...               1216.0  
3  MKKFLTISILAGLILSLPLSAFAYSNPFKLTNSWKWDTGVFYGEGD...               1213.0  
4  MTMAIMVSFILSLPLTTFAFSNPIKLPNSWEWDTGVFYGEGDPYIL...               1211.0  
5  MSLWLSLPYAALAYSNPMDLPESWTWDSGEYYGEGDPYILKHNGIY...               1209.0  
6  MRFVRVLLIMGGLFIFPSVSFAYSNPFDLSDSWEWDNGDFYGEGDP...               1209.0  
7  MTTVLASFLLSIPLSAFAYSNPLTLSDSWKWDTGVFYGEGDPYILK...               1207.0  
8  MSLWLSLPYAALAYSNPMDLPESWTWDSGEYYGEGDPYILKHNGIY...               1207.0  
9  MKVKKLVASLTAAGCLSGMVTALPQLVLNSYAAEIVYNDFEISYEG...               1155.0

In [ ]:
# Don't delete this cell


In [ ]:
# Don't delete this cell


In [ ]:
# Don't delete this cell
